In [1]:
from pinecone import Pinecone as PineconeClient
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda


In [3]:
import os
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = "aws"
PINECONE_INDEX_NAME ="build-a-product"

In [4]:
pinecone = PineconeClient(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

## RAG Prompt

In [6]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
chain.invoke("what are minimum viable products?")

"Minimum viable products (MVPs) are a version of a new product that allows a team to collect the maximum amount of validated learning about customers with the least effort. The concept of an MVP is to build something small, typically a fraction of the total envisioned product, to test and validate the core value propositions before investing significant resources. It's a way to ensure that the product being developed is valuable to the market and has the potential to grow. The MVP approach encourages starting with the essentials to satisfy early adopters and then iterating based on feedback, rather than attempting to launch with a fully-featured product. This strategy is often used in software development but can be applied to other products or services as well."

In [8]:
chain.invoke("what is the product company gap and how can it be solved?")

'The "Product Company Gap" refers to the challenge of transitioning from having a product idea to building a successful company around that product. It involves considerations beyond the product itself, such as go-to-market strategies, pricing, and overall business model design that are essential for scaling up and establishing a viable business.\n\nTo solve the Product Company Gap, the documents suggest:\n\n1. Designing products with go-to-market fit in mind, meaning that products should be designed not only to meet customer needs but also to be easily sellable.\n2. Architecting a business model from the beginning that supports scalability, which includes thoughtful pricing strategies and other elements that facilitate sales and growth.\n3. Starting with a minimum viable segment to establish product-market fit, then expanding gradually to broader markets.\n4. Continuously thinking about the business model from day one, rather than treating it as an afterthought, to ensure that the pro